# BoxSort for Sorting a Square Matricies

This notebook is intended to show what and how the *BoxSort* class works.

*BoxSort* creates the sorted matricies that are cut by [BoxCut class](https://github.com/peterwinter/boxcluster_tutorial/blob/master/boxcut.ipynb) into several "boxes".

The diagonal is a line of symetry. This method optimizes position so high affinity positions are closest to the line of symetry. ie. every node is closest to it's strongest connections. ie. the matrix is sorted.

The key classes in this process include:

- **np.array** -- the data structure 
- **OrderedArray** -- a boxcluster class specialized for symetrical rearangements
- **BoxSort** -- a class for simulated annealing using the **OrderedArray** class

On the surface this seems complicated, so an example might help.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# global imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# the boxcluster package
from boxcluster import BoxSort
from boxcluster.fake_data import generate_nested_data

In [ ]:
# local imports
from plotting import mplot

### Load Sample Data

To test out this function we load a sample dataset that contains the type of structures we are looking for.
The data is represented as a square 64 x 64 *numpy array* with values between 0 and 1.

It's a numpy array, we're just plotting it visually here.

In [ ]:
test_soln = generate_nested_data(noise=0.05)
print('Solution')
mplot(test_soln)
plt.show()

### Make Test Set

Shuffle the rows so that the algorithm can try to reassemble the pieces.

In [ ]:
n = len(test_soln)
order = np.arange(n)
np.random.shuffle(order)
test = test_soln[:, order][order, :]
print('Test')
mplot(test)

# BoxSort Example


In [ ]:
# initialize the instance
bs = BoxSort(test)

# returns a OrderedArray class object with the solution.
result = bs(save_history=True)

# show result
mplot(result.matrix)
plt.show()

# Under The Hood

### Introspecting History

Because we ran **bc** using while saving history, `bc(save_history=True)`, we've got a list of namedtuples (from collections) that let us introspect how the simulated annealing algorithm ran. 

Each of the named tuple (called a *trace*) contains seven elements:
- evals (a counter)
- last_move (turns since move was accepted)
- move_accepted (boolean, some moves are stored but not accepted)
- temp (current temperature)
- current_fit (fitness/energy of current state)
- new_fit (fitness/energy of proposed move)

In [ ]:
history = bs.history
history[:3]

This can conveniently turned into a dataframe and used for plotting

In [ ]:
df = pd.DataFrame(history).set_index('evals')
df.head()

### Maximizing Value along diagonal

We can check how well we maximize this fitness function

In [ ]:
df[['current_fit', 'new_fit']].plot(alpha=0.5)

### Halting Condition

Examine all the properties involved with progressing for this anneal.

In [ ]:
df['last_move'].plot(alpha=0.5)

In [ ]:
df['temp'].plot()

In [ ]:
df['moves_this_temp'].plot(alpha=0.5)